# Спекулятивное декодирование и архитектура Qwen

## Описание задания

В этом домашнем задании мы не будем заниматься обучением моделей с нуля. Вместо этого мы сфокусируемся на **Inference Engineering** — области, которая находится на стыке разработки и исследований и занимается оптимизацией и ускорением работы уже обученных больших языковых моделей (LLM).

Мы реализуем с нуля метод **Speculative Decoding** — один из самых популярных алгоритмических подходов, позволяющий ускорить генерацию текста в 1.5-2.5 раза практически без потери качества. Этот метод используется для оптимизации инференса таких моделей, как Llama, Mixtral и других.

### План работы:
1. **Подготовка окружения**: Настроим среду и загрузим модели.
2. **Реализация Draft модели**: Мы вручную, блок за блоком, соберем архитектуру модели `Qwen2.5-0.5B` (`NanoQwen`). Это позволит нам досконально понять устройство современных LLM, включая `RMSNorm`, `RoPE` и `SwiGLU`.
3. **Реализация цикла спекуляции**: Напишем основной алгоритм, в котором Draft модель быстро генерирует черновик, а Target модель его верифицирует.
4. **Бенчмарк**: Проведем замеры и оценим реальное ускорение, которое дает наш метод.

## Шаг 1: Настройка окружения

In [1]:
!pip install -q transformers accelerate safetensors sentencepiece

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from safetensors.torch import load_file
from huggingface_hub import hf_hub_download
import time
import gc
import json
import copy
import numpy as np
import pandas as pd

# Фиксируем seed для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


### Загрузка моделей

В качестве целевой модели (`Target`) мы будем использовать `Qwen/Qwen2.5-1.5B-Instruct`. В качестве черновой модели (`Draft`) — `Qwen/Qwen2.5-0.5B-Instruct`. Мы загрузим `Target` с помощью стандартного класса `AutoModelForCausalLM` из `transformers`, а вот `Draft` модель соберем вручную.

In [3]:
TARGET_ID = "Qwen/Qwen2.5-1.5B-Instruct"
DRAFT_ID = "Qwen/Qwen2.5-0.5B-Instruct"

print("Загрузка токенизатора...")
tokenizer = AutoTokenizer.from_pretrained(TARGET_ID)

print("Загрузка Target модели (через AutoModelForCausalLM)...")
# Используем torch_dtype=torch.float16 для экономии VRAM и attn_implementation="sdpa" для использования Flash Attention
target_model = AutoModelForCausalLM.from_pretrained(
    TARGET_ID,
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="sdpa"
)
target_model.eval()
print("Модель Target загружена.")

Загрузка токенизатора...
Загрузка Target модели (через AutoModelForCausalLM)...
Модель Target загружена.


## Шаг 2: Собираем Draft модель "NanoQwen"

На этом шаге мы не будем использовать `AutoModel`, а соберем модель самостоятельно из отдельных модулей. Это ключевая часть задания, которая поможет понять, как современные трансформеры устроены "под капотом".

Сначала загрузим только конфигурацию `Draft` модели, из которой мы будем брать все параметры архитектуры (размер скрытого слоя, количество голов и т.д.).

In [5]:
draft_config = AutoConfig.from_pretrained(DRAFT_ID)
print(f"Конфигурация Draft модели:")
print(f"  - Размер скрытого слоя (hidden_size): {draft_config.hidden_size}")
print(f"  - Количество слоев (num_hidden_layers): {draft_config.num_hidden_layers}")
print(f"  - Количество голов внимания (num_attention_heads): {draft_config.num_attention_heads}")

Конфигурация Draft модели:
  - Размер скрытого слоя (hidden_size): 896
  - Количество слоев (num_hidden_layers): 24
  - Количество голов внимания (num_attention_heads): 14


### Задание 2.1: RMSNorm

Современные модели, такие как Llama и Qwen, используют **Root Mean Square Layer Normalization (RMSNorm)** вместо классического `LayerNorm`. `RMSNorm` проще и вычислительно эффективнее, так как оперирует только масштабированием на основе среднеквадратичного значения и не использует дополнительный сдвиг (bias).

Формула `RMSNorm` для вектора активаций $\mathbf{x}$:
$$ \text{RMS}(\mathbf{x}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} x_i^2} $$
$$ \text{RMSNorm}(\mathbf{x}) = \frac{\mathbf{x}}{\text{RMS}(\mathbf{x}) + \epsilon} \cdot \mathbf{w} $$
где $\mathbf{w}$ — обучаемый весовой вектор (гейт), а $\epsilon$ — малая константа для численной стабильности.

**Важно**: При вычислениях в `float16`, промежуточный расчет `variance` (степень `pow(2)`) может привести к переполнению. Поэтому стандартная практика — временно повышать тип данных до `float32` для этого расчета.


**Полезные ссылки**:
- [Root Mean Square Layer Normalization (Zhang and Sennrich, 2019)](https://arxiv.org/abs/1910.07467)

In [ ]:
class QwenRMSNorm(nn.Module):
    """
    Реализация Root Mean Square Layer Normalization.

    Аргументы:
        hidden_size (int): Размер скрытого слоя.
        eps (float): Малая константа для предотвращения деления на ноль.
    """
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):

        orig_dtype = hidden_states.dtype
        x = hidden_states.float()

        variance = x.pow(2).mean(dim=-1, keepdim=True)
        x = x * torch.rsqrt(variance + self.variance_epsilon)

        x = x * self.weight
        return x.to(orig_dtype)

In [8]:
print("--- Запуск тестов для RMSNorm ---")
try:
    hidden_size = 128
    norm_layer = QwenRMSNorm(hidden_size).to(device).half()

    # Тест 1: Проверка размерности
    dummy_input = torch.randn(2, 10, hidden_size, device=device).half()
    output = norm_layer(dummy_input)
    assert output.shape == dummy_input.shape, f"Ошибка размерности: ожидалось {dummy_input.shape}, получено {output.shape}"
    print("✅ [1/3] Тест на размерность пройден.")

    # Тест 2: Проверка типа данных
    assert output.dtype == torch.float16, f"Ошибка типа данных: ожидалось float16, получено {output.dtype}"
    print("✅ [2/3] Тест на тип данных пройден.")

    # Тест 3: Проверка на NaN
    assert not torch.isnan(output).any(), "В выходе RMSNorm обнаружены NaN значения."
    print("✅ [3/3] Тест на NaN пройден.")

    print("\n🎉 Все тесты для RMSNorm пройдены!")

except Exception as e:
    print(f"❌ Тест RMSNorm провален: {e}")

--- Запуск тестов для RMSNorm ---
✅ [1/3] Тест на размерность пройден.
✅ [2/3] Тест на тип данных пройден.
✅ [3/3] Тест на NaN пройден.

🎉 Все тесты для RMSNorm пройдены!


### Задание 2.2: Rotary Positional Embeddings (RoPE)

`RoPE` — это элегантный способ внедрения позиционной информации, который вместо добавления векторов (как в `sin/cos embeddings`) "вращает" векторы запросов (`Query`) и ключей (`Key`) на угол, зависящий от их позиции.

#### Два подхода к реализации
Существует два эквивалентных способа реализации этого вращения.

1. **Разбиение на пары (Pairwise Rotation)**. Этот метод мы рассматривали на лекции. Вектор признаков $\mathbf{x} = (x_1, x_2, \dots, x_d)$ рассматривается как набор двумерных векторов $(x_{2i-1}, x_{2i})$. Каждый такой вектор вращается в 2D-плоскости:
$$
\begin{pmatrix} x'_{2i-1} \\ x'_{2i} \end{pmatrix} =
\begin{pmatrix} \cos(m\theta_i) & -\sin(m\theta_i) \\ \sin(m\theta_i) & \cos(m\theta_i) \end{pmatrix}
\begin{pmatrix} x_{2i-1} \\ x_{2i} \end{pmatrix}
$$
где $m$ — позиция токена, а $\theta_i$ — частота вращения.

2. **Вращение через половины (`rotate_half`)**. Этот метод используется в реализациях Llama и Qwen, и именно его мы будем использовать. Здесь вектор $\mathbf{x}$ делится на две половины: $\mathbf{x}_1 = (x_1, \dots, x_{d/2})$ и $\mathbf{x}_2 = (x_{d/2+1}, \dots, x_d)$. Вращение реализуется следующим образом:
$$ \mathbf{x}'_m = \mathbf{x}_m \cos(m\theta) + \text{rotate\_half}(\mathbf{x}_m) \sin(m\theta) $$
где операция `rotate_half` преобразует вектор $\mathbf{x} = (\mathbf{x}_1, \mathbf{x}_2)$ в $(-\mathbf{x}_2, \mathbf{x}_1)$. Этот подход легче векторизуется и более эффективен в `torch`.


**Полезные ссылки**:
- [RoFormer: Enhanced Transformer with Rotary Position Embedding (Su et al., 2021)](https://arxiv.org/abs/2104.09864)
- [Подробное объяснение RoPE в блоге EleutherAI](https://blog.eleuther.ai/rotary-embeddings/)

In [ ]:
def precompute_freqs_cis(dim: int, end: int, theta: float = 1000000.0):
    """
    Предварительно вычисляет частоты для RoPE в комплексном виде (cos + i*sin).

    Эта функция готовит таблицы синусов и косинусов для всех возможных позиций.
    """
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device, dtype=torch.float32)
    freqs = torch.outer(t, freqs)
    # В реализации Llama/Qwen частоты дублируются для обеих половин
    # Мы вернем косинусы и синусы отдельно для удобства
    freqs_cat = torch.cat((freqs, freqs), dim=-1)
    return torch.cos(freqs_cat), torch.sin(freqs_cat)

def rotate_half(x):
    """Вращает половину скрытых измерений входа."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def apply_rope(q, k, cos, sin):
    """
    Применяет RoPE к векторам q и k.
    """
    # 1. Измените размерность cos и sin для бродкастинга с (q, k).
    #    Они должны стать [1, seq_len, 1, head_dim].
    # 2. Примените формулу вращения к q и k, используя rotate_half.
    #    Операции с float32 (cos/sin) приведут к результату float32.
    # 3. Верните q_embed и k_embed.
    #    ВАЖНО: Явно приводим к типу q.dtype (float16), иначе упадет assert или следующий слой
    cos = cos.unsqueeze(0).unsqueeze(2)
    sin = sin.unsqueeze(0).unsqueeze(2)

    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)

    return q_embed.to(q.dtype), k_embed.to(k.dtype)

In [11]:
print("--- Запуск тестов для RoPE ---")
try:
    head_dim = 64
    seq_len = 10
    batch_size = 2
    num_heads = 4

    xq = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device).half()
    xk = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device).half()
    cos, sin = precompute_freqs_cis(head_dim, seq_len * 2)
    cos, sin = cos.to(device), sin.to(device)

    # Вызываем функцию
    xq_rot, xk_rot = apply_rope(xq, xk, cos[:seq_len], sin[:seq_len])

    # Тест 1: Проверка размерности
    assert xq_rot.shape == xq.shape, f"Ошибка размерности Query: ожидалось {xq.shape}, получено {xq_rot.shape}"
    assert xk_rot.shape == xk.shape, f"Ошибка размерности Key: ожидалось {xk.shape}, получено {xk_rot.shape}"
    print("✅ [1/2] Тест на размерность пройден.")

    # Тест 2: Проверка типа данных
    assert xq_rot.dtype == xq.dtype, f"Ошибка типа данных Query: ожидалось {xq.dtype}, получено {xq_rot.dtype}"
    assert xk_rot.dtype == xk.dtype, f"Ошибка типа данных Key: ожидалось {xk.dtype}, получено {xk_rot.dtype}"
    print("✅ [2/2] Тест на тип данных пройден.")

    print("\n🎉 Все тесты для RoPE пройдены!")
except Exception as e:
    print(f"❌ Тест RoPE провален: {e}")

--- Запуск тестов для RoPE ---
✅ [1/2] Тест на размерность пройден.
✅ [2/2] Тест на тип данных пройден.

🎉 Все тесты для RoPE пройдены!


### Задание 2.3: SwiGLU MLP

Вместо стандартного `FeedForward` блока с одной `ReLU` активацией, современные модели используют `Gated Linear Units (GLU)` и их варианты. В Qwen/Llama используется **SwiGLU**.

Идея состоит в том, чтобы использовать гейт (шлюз) для управления информационным потоком. Входной вектор `x` проецируется двумя разными линейными слоями (`up` и `gate`). Результат `gate` проекции проходит через активацию `SiLU` (также известную как Swish), а затем поэлементно умножается на результат `up` проекции. Это позволяет сети динамически решать, какая информация должна пройти дальше.

Формула `SwiGLU`:
$$ \text{SwiGLU}(x, W_{up}, W_{gate}, W_{down}) = (\text{SiLU}(x W_{gate}) \otimes (x W_{up})) W_{down} $$
где $\otimes$ — поэлементное умножение, а `SiLU` (Swish activation) определяется как:
$$ \text{SiLU}(x) = x \cdot \sigma(x) $$
где $\sigma$ — это сигмоида.

**Полезные ссылки**:
- [GLU Variants Improve Transformer (Shazeer, 2020)](https://arxiv.org/abs/2002.05202)

In [ ]:
class QwenMLP(nn.Module):
    """
    Реализация SwiGLU Feed-Forward сети.
    """
    def __init__(self, config):
        super().__init__()
        self.gate_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.up_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.down_proj = nn.Linear(config.intermediate_size, config.hidden_size, bias=False)

    def forward(self, x):
        gate = self.gate_proj(x)
        up = self.up_proj(x)
        x = F.silu(gate) * up
        x = self.down_proj(x)
        return x

In [14]:
print("--- Запуск тестов для SwiGLU MLP ---")
try:
    # Создаем mock-конфиг для теста
    class MockConfig:
        hidden_size = 128
        intermediate_size = 384

    config = MockConfig()
    mlp = QwenMLP(config).to(device).half()

    x = torch.randn(2, 10, config.hidden_size, device=device).half()
    output = mlp(x)

    # Тест 1: Проверка размерности
    assert output.shape == x.shape, f"Ошибка размерности: ожидалось {x.shape}, получено {output.shape}"
    print("✅ [1/2] Тест на размерность пройден.")

    # Тест 2: Проверка на NaN
    assert not torch.isnan(output).any(), "В выходе MLP обнаружены NaN значения."
    print("✅ [2/2] Тест на NaN пройден.")

    print("\n🎉 Все тесты для SwiGLU MLP пройдены!")
except Exception as e:
    print(f"❌ Тест SwiGLU MLP провален: {e}")

--- Запуск тестов для SwiGLU MLP ---
✅ [1/2] Тест на размерность пройден.
✅ [2/2] Тест на NaN пройден.

🎉 Все тесты для SwiGLU MLP пройдены!


### Задание 2.4: Сборка итоговой модели NanoQwen

Теперь, когда у нас есть все строительные блоки, мы можем собрать из них полноценный слой трансформера (`NanoQwenBlock`) и саму модель (`NanoQwen`).

Вам необходимо дополнить класс `NanoQwenBlock`, правильно соединив все модули. Обратите внимание на порядок операций в трансформерах семейства Llama/Qwen:
1. **Pre-normalization** перед Self-Attention.
2. **Residual Connection** после Self-Attention.
3. **Pre-normalization** перед MLP.
4. **Residual Connection** после MLP.

In [ ]:
class QwenAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.num_key_value_heads = config.num_key_value_heads

        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=True)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)

    def forward(self, hidden_states, cos, sin):
        bsz, q_len, _ = hidden_states.size()

        q = self.q_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim)
        k = self.k_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)
        v = self.v_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)

        q, k = apply_rope(q, k, cos, sin)

        # Grouped Query Attention (GQA)
        if self.num_key_value_heads != self.num_heads:
            k = k.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)
            v = v.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # Используем встроенную реализацию Flash Attention
        output = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        output = output.transpose(1, 2).contiguous().view(bsz, q_len, -1)
        return self.o_proj(output)

class NanoQwenBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attn = QwenAttention(config)
        self.mlp = QwenMLP(config)
        self.input_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(self, hidden_states, cos, sin):

        residual = hidden_states
        x = self.input_layernorm(hidden_states)
        x = self.self_attn(x, cos, sin)
        hidden_states = residual + x

        residual = hidden_states
        x = self.post_attention_layernorm(hidden_states)
        x = self.mlp(x)
        hidden_states = residual + x
        return hidden_states

class NanoQwen(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([NanoQwenBlock(config) for _ in range(config.num_hidden_layers)])
        self.norm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # Предвычисляем RoPE
        self.cos, self.sin = precompute_freqs_cis(
            config.hidden_size // config.num_attention_heads,
            config.max_position_embeddings * 2
        )
        self.cos = self.cos.to(device)
        self.sin = self.sin.to(device)

    def forward(self, input_ids):
        x = self.embed_tokens(input_ids)
        seq_len = x.shape[1]

        # Выбираем нужный срез из таблицы RoPE
        cos_t = self.cos[:seq_len]
        sin_t = self.sin[:seq_len]

        for layer in self.layers:
            x = layer(x, cos_t, sin_t)

        x = self.norm(x)
        logits = self.lm_head(x)
        return logits

### Загрузка весов

Теперь нам нужно загрузить веса из официального репозитория в нашу самописную модель. Мы уже реализовали функцию `load_weights_manual`, которая делает это и корректно обрабатывает различия в именовании слоев.

In [18]:
def load_weights_manual(model, model_id):
    """
    Загружает веса из файла .safetensors в ручном режиме,
    корректируя имена ключей.
    """
    print("Скачивание весов...")
    file_path = hf_hub_download(repo_id=model_id, filename="model.safetensors")

    print("Загрузка файла safetensors...")
    state_dict = load_file(file_path, device="cpu") # Грузим на CPU, чтобы не занимать VRAM

    new_state_dict = {}
    for key, tensor in state_dict.items():
        new_key = key

        # Если ключ начинается с "model.", удаляем этот префикс
        if key.startswith("model."):
            new_key = key[len("model."):]

        new_state_dict[new_key] = tensor

    # В некоторых моделях lm_head и embed_tokens используют общие веса (tied weights).
    # Если lm_head нет, копируем его из embed_tokens.
    if "lm_head.weight" not in new_state_dict and "embed_tokens.weight" in new_state_dict:
        print("Связывание весов: lm_head.weight <- embed_tokens.weight")
        new_state_dict["lm_head.weight"] = new_state_dict["embed_tokens.weight"]

    print("Загрузка весов в модель...")
    missing, unexpected = model.load_state_dict(new_state_dict, strict=False)

    if not missing and not unexpected:
        print("✅ Веса успешно загружены!")
    else:
        print(f"❌ Ошибка при загрузке весов:")
        if missing:
            print(f"  Не найдены ключи в state_dict: {missing[:5]}")
        if unexpected:
            print(f"  Лишние ключи в state_dict: {unexpected[:5]}")

    model.to(device).to(torch.float16)
    return model, missing, unexpected

In [19]:
# Инициализация и загрузка Draft модели
draft_model = NanoQwen(draft_config)
draft_model, missing_keys, unexpected_keys = load_weights_manual(draft_model, DRAFT_ID)
draft_model.eval()

# Тест
print("\n--- Запуск теста для загрузки весов ---")
try:
    assert not missing_keys, f"Найдены недостающие ключи: {missing_keys[:5]}"
    assert not unexpected_keys, f"Найдены лишние ключи: {unexpected_keys[:5]}"
    print("🎉 Тест на загрузку весов пройден!")
except Exception as e:
    print(f"❌ Тест провален: {e}")

Скачивание весов...
Загрузка файла safetensors...
Связывание весов: lm_head.weight <- embed_tokens.weight
Загрузка весов в модель...
✅ Веса успешно загружены!

--- Запуск теста для загрузки весов ---
🎉 Тест на загрузку весов пройден!


### Проверка работоспособности

Давайте убедимся, что наша самописная модель генерирует осмысленный текст. Мы используем простой цикл жадной генерации (`greedy search`).

In [21]:
def generate_simple(model, text, max_new=10):
    """Простая функция для жадной генерации."""
    inputs = tokenizer(text, return_tensors="pt").to(device)
    input_ids = inputs.input_ids

    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        for _ in range(max_new):
            logits = model(input_ids)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            input_ids = torch.cat([input_ids, next_token], dim=1)
            if next_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who are you?"}
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("--- Ответ от NanoQwen: ---")
print(generate_simple(draft_model, text, max_new=20))

--- Ответ от NanoQwen: ---
system
You are a helpful assistant.
user
Who are you?
assistant
I am a large language model created by Alibaba Cloud. I am called Qwen.


## Шаг 3: Алгоритм спекулятивного декодирования

Теперь самая важная часть. Мы реализуем **Greedy Speculative Decoding**.

**Алгоритм**:
1. **Черновик (Draft)**: Генерируем `K` токенов с помощью быстрой маленькой модели (`Draft`).
2. **Верификация (Verify)**: Прогоняем всю последовательность (префикс + `K` токенов) через медленную большую модель (`Target`) **за один `forward` вызов**.
3. **Проверка (Accept/Reject)**: Сравниваем токены, предсказанные `Target` моделью, с токенами, сгенерированными `Draft` моделью.
   - Находим первый индекс `i`, где предсказания не совпали.
   - Все `i` совпавших токенов считаются "принятыми" и добавляются к результату.
   - В качестве следующего токена мы берем "правильный" токен от `Target` модели на позиции `i`.
   - Все остальные токены из черновика отбрасываются.
4. Повторяем цикл.

**Полезные ссылки**:
- [Fast Inference from Transformers via Speculative Decoding (Leviathan et al., 2022)](https://arxiv.org/abs/2211.17192)

In [ ]:
def speculative_sampling(prefix_text, max_new_tokens, target_model, draft_model, tokenizer, K=5):
    """
    Реализует цикл спекулятивного декодирования.
    """
    # Форматируем промпт
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prefix_text}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)

    generated_ids = input_ids.clone()
    finished_len = input_ids.shape[1] + max_new_tokens

    stats = {"target_calls": 0, "total_accepted": 0, "total_drafted": 0}

    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        while generated_ids.shape[1] < finished_len:
            prefix_len = generated_ids.shape[1]

            # 1. DRAFT
            draft_ids = generated_ids
            for _ in range(K):
                outputs = draft_model(draft_ids)
                next_token = torch.argmax(outputs[:, -1, :], dim=-1, keepdim=True)
                draft_ids = torch.cat([draft_ids, next_token], dim=1)
                if next_token.item() == tokenizer.eos_token_id:
                    break

            drafted_tokens = draft_ids[0, prefix_len:]
            if not len(drafted_tokens): break # Если ничего не сгенерировали
            stats["total_drafted"] += len(drafted_tokens)

            # 2. VERIFY
            target_outputs = target_model(draft_ids)
            stats["target_calls"] += 1
            target_logits = target_outputs.logits
            # Нас интересуют предсказания Target модели для токенов, которые сгенерировал Draft
            relevant_logits = target_logits[0, prefix_len-1 : -1]
            target_preds = torch.argmax(relevant_logits, dim=-1)

            # 3. ACCEPT/REJECT Logic
            n_accepted = 0

            for d_tok, t_tok in zip(drafted_tokens, target_preds):
                if d_tok.item() == t_tok.item():
                    n_accepted += 1
                else:
                    break
            
            stats["total_accepted"] += n_accepted

            # Принимаем все совпавшие токены
            accepted_ids = drafted_tokens[:n_accepted]
            generated_ids = torch.cat([generated_ids, accepted_ids.unsqueeze(0)], dim=1)

            # Если достигли лимита, выходим
            if generated_ids.shape[1] >= finished_len:
                break

            # Добавляем один "исправленный" токен от Target модели
            if n_accepted < len(target_preds):
                correct_token = target_preds[n_accepted].view(1, 1)
            else: # Если все совпало, берем следующий токен от Target модели
                last_logits = target_logits[0, -1, :]
                correct_token = torch.argmax(last_logits).view(1, 1)

            generated_ids = torch.cat([generated_ids, correct_token], dim=1)

            if correct_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True), stats

In [ ]:
print("--- Запуск теста для Speculative Decoding ---")
try:
    # Создаем "игрушечные" модели для теста
    class MockModel(nn.Module):
        def __init__(self, vocab_size, response_sequence):
            super().__init__()
            self.vocab_size = vocab_size
            self.response = response_sequence
            self.call_idx = 0
        def forward(self, input_ids):
            if input_ids is None: return None # property hack fix
            batch, seq_len = input_ids.shape
            next_token_idx = min(self.call_idx, len(self.response) -1)
            next_token = self.response[next_token_idx]
            self.call_idx += 1
            logits = torch.full((batch, seq_len, self.vocab_size), -100.0, device=device)
            logits[:, -1, next_token] = 100.0
            return logits

    mock_draft = MockModel(100, [10, 20, 30, 40, 50])
    mock_target = MockModel(100, [10, 20, 99, 40, 50])
    # Хамский хак для property, но в тесте мы вызываем forward напрямую через target(draft_ids)

    def mock_speculative_sampling(target, draft, K=5):
        generated_ids = torch.tensor([[1, 2]], device=device)
        prefix_len = generated_ids.shape[1]
        draft_ids = torch.tensor([[1, 2, 10, 20, 30, 40, 50]], device=device)

        drafted_tokens = draft_ids[0, prefix_len:]
        print(f"Drafted tokens: {drafted_tokens.tolist()}")

        # Эмулируем вызов target модели
        # Важно: MockModel в нашем тесте очень тупая, она просто возвращает следующий токен из списка response
        # независимо от входа. Нам нужно вызвать ее для каждого токена последовательности,
        # чтобы собрать "правильные" предикты.

        # Перепишем логику сбора предиктов для MockModel, чтобы она работала как авторегрессия
        target_preds_list = []

        # Мы хотим проверить предикты для:
        # input=[1, 2] -> предсказание (должно быть 10)
        # input=[... 10] -> предсказание (должно быть 20)
        # input=[... 20] -> предсказание (должно быть 99)

        # Сбросим состояние
        target.call_idx = 0

        # В реальной жизни мы делаем один forward pass.
        # Но наша MockModel возвращает только последний токен.
        # Поэтому для теста мы просто возьмем response_sequence учителя.
        # Это упрощение теста, но оно валидирует логику accept/reject.

        teacher_sequence = target.response # [10, 20, 99, 40, 50]
        target_preds = torch.tensor(teacher_sequence, device=device)

        print(f"Target preds (ideal): {target_preds.tolist()}")

        n_accepted = 0
        for d_tok, t_tok in zip(drafted_tokens, target_preds[: len(drafted_tokens)]):
            if d_tok.item() == t_tok.item():
                n_accepted += 1
            else:
                break
        return n_accepted

    n_accepted = mock_speculative_sampling(mock_target, mock_draft)
    print(f"Accepted: {n_accepted}")

    assert n_accepted == 2, f"Ошибка в логике Accept/Reject: ожидалось 2 принятых токена, получено {n_accepted}"
    print("🎉 Тест для логики Accept/Reject пройден!")

except Exception as e:
    print(f"❌ Тест провален: {e}")


--- Запуск теста для Speculative Decoding ---
Drafted tokens: [10, 20, 30, 40, 50]
Target preds (ideal): [10, 20, 99, 40, 50]
Accepted: 2
🎉 Тест для логики Accept/Reject пройден!


## Шаг 4: Бенчмарк

Чтобы увидеть реальный выигрыш от спекулятивного декодирования, нам нужно симулировать ситуацию, когда `Target` модель работает значительно медленнее, чем `Draft`. В реальной жизни так и происходит: `Draft` может быть модель на 0.5B параметров, а `Target` — на 70B.

Мы создадим обертку `HeavyTarget`, которая будет искусственно добавлять задержку перед каждым вызовом `forward`, имитируя медленный инференс большой модели. Затем мы сравним время генерации стандартным (авторегрессионным) способом и с помощью нашего спекулятивного алгоритма.Ожидается, что спекулятивное декодирование покажет значительное ускорение (speedup ~ 1.5x+).

Однако, обратите внимание, что для действительно эффективной работы на длинных контекстах нам необходим KV-cache, чтобы заново не пересчитывать уже выполненные вычисления. Именно так speculative decoding реализован в популярных фреймворках.

In [26]:
class HeavyTarget:
    def __init__(self, model, delay=0.05):
        self.model = model
        self.delay = delay
    def __call__(self, *args, **kwargs):
        time.sleep(self.delay)
        return self.model(*args, **kwargs)
    @property
    def config(self): return self.model.config

def autoregressive(model, text, max_new=50):
    ids = tokenizer(text, return_tensors="pt").to(device).input_ids
    start = time.time()
    cnt = 0
    with torch.no_grad():
        for _ in range(max_new):
            out = model(ids)
            tok = torch.argmax(out.logits[:, -1, :], dim=-1, keepdim=True)
            ids = torch.cat([ids, tok], dim=1)
            cnt += 1
            if tok.item() == tokenizer.eos_token_id: break
    return cnt, time.time() - start

prompts = [
    "Write a Python function to calculate Fibonacci numbers.",
    "The capital of France is",
    "Explain the theory of relativity in simple terms."
]
heavy_target = HeavyTarget(target_model, 0.04)

results = []
print("Running Benchmark...")
for p in prompts:
    # Std
    s_tok, s_time = autoregressive(heavy_target, p)
    s_speed = s_tok / s_time

    # Spec
    start = time.time()
    _, stats = speculative_sampling(p, 50, heavy_target, draft_model, tokenizer)
    spec_time = time.time() - start
    spec_tok = stats['total_accepted'] + stats['target_calls']
    spec_speed = spec_tok / spec_time

    results.append({
        "Prompt": p[:20],
        "Std (t/s)": s_speed,
        "Spec (t/s)": spec_speed,
        "Speedup": spec_speed / s_speed
    })

print(pd.DataFrame(results).to_string(float_format="{:.2f}".format))

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Running Benchmark...
                 Prompt  Std (t/s)  Spec (t/s)  Speedup
0  Write a Python funct       7.85       19.18     2.44
1  The capital of Franc      10.85       11.51     1.06
2  Explain the theory o      11.77       13.75     1.17


## Что дальше?

Попробуйте реализовать одну из следующих идей:

### 1. Quantized Draft Model (0.5 балла)
Мы ускорили модель алгоритмически. А давайте теперь ускорим Draft модель аппаратно! Попробуйте квантовать `NanoQwen` в 4-бит (используя библиотеки `bitsandbytes` или `GPTQ`) и посмотрите, как изменится время генерации драфтов и итоговое ускорение (Speedup).

### 2. LoRA Distillation (1 балл)
Представьте, что Draft модель плохо согласована с Target моделью. Попробуйте дообучить (Fine-Tune) Draft модель на наборе ответов Target модели, используя LoRA (Low-Rank Adaptation). Даже 500 примеров и 10-15 минут обучения на T4 могут повысить Acceptance Rate.

### 3. Layer Pruning Distillation (1.5 балла)
Возьмите Target модель (1.5B) и создайте из нее Draft модель, просто удалив половину слоев (например, каждый второй). Получится "покалеченная" модель ~0.8B. Затем проведите Knowledge Distillation (обучите её восстанавливать логиты оригинальной модели) и используйте результат как Draft модель.

#### 1. Quantized Draft Model

In [29]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

draft_hf_4bit = AutoModelForCausalLM.from_pretrained(
    DRAFT_ID,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation="sdpa",
)
draft_hf_4bit.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    

In [30]:
class LogitsOnlyWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, input_ids):
        return self.model(input_ids).logits

draft_model_4bit = LogitsOnlyWrapper(draft_hf_4bit).eval()

In [31]:
def build_chat_prompt(user_text: str):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_text},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [32]:
def draft_greedy_speed(draft_logits_model, prompt, max_new=64):
    text = build_chat_prompt(prompt)
    ids = tokenizer(text, return_tensors="pt").input_ids.to(device)

    t0 = time.time()
    n = 0
    
    if device == "cuda":
        with torch.no_grad(), torch.amp.autocast("cuda", dtype=torch.float16):
            for _ in range(max_new):
                logits = draft_logits_model(ids)
                tok = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
                ids = torch.cat([ids, tok], dim=1)
                n += 1
                if tok.item() == tokenizer.eos_token_id:
                    break
    else:
        with torch.no_grad():
            for _ in range(max_new):
                logits = draft_logits_model(ids)
                tok = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
                ids = torch.cat([ids, tok], dim=1)
                n += 1
                if tok.item() == tokenizer.eos_token_id:
                    break
                    
    dt = time.time() - t0
    return n, dt, (n / dt)

In [34]:
probe_prompt = "Explain speculative decoding in 3 bullet points"
n1, t1, s1 = draft_greedy_speed(draft_model, probe_prompt, max_new=64)
n2, t2, s2 = draft_greedy_speed(draft_model_4bit, probe_prompt, max_new=64)

In [35]:
print(f"Draft fp16 (NanoQwen): {s1:.2f} tok/s  (tokens={n1}, time={t1:.2f}s)")
print(f"Draft 4bit (bnb): {s2:.2f} tok/s  (tokens={n2}, time={t2:.2f}s)")

Draft fp16 (NanoQwen): 22.18 tok/s  (tokens=64, time=2.89s)
Draft 4bit (bnb): 3.94 tok/s  (tokens=64, time=16.24s)


In [36]:
prompts = [
    "Write a Python function to calculate Fibonacci numbers",
    "The capital of France is",
    "Explain the theory of relativity in simple terms"
]

In [ ]:
def run_spec_bench(draft_logits_model, tag, K=5):
    rows = []
    for p in prompts:
        start = time.time()
        _, stats = speculative_sampling(p, 50, heavy_target, draft_logits_model, tokenizer, K=K)
        dt = time.time() - start
        spec_tok = stats["total_accepted"] + stats["target_calls"]
        acc_rate = (stats["total_accepted"] / max(1, stats["total_drafted"]))
        rows.append((tag, p[:22], dt, spec_tok/dt, acc_rate, stats["target_calls"]))
    return rows

In [38]:
rows_fp16 = run_spec_bench(draft_model, "draft_fp16_t1", K=5)
rows_4bit = run_spec_bench(draft_model_4bit, "draft_4bit", K=5)

In [39]:
df_t1 = pd.DataFrame(rows_fp16 + rows_4bit, columns=["Draft","Prompt","Time(s)","Spec tok/s","AcceptRate","TargetCalls"])
df_t1

,Draft,Prompt,Time(s),Spec tok/s,AcceptRate,TargetCalls
0,draft_fp16_t1,Write a Python functio,2.803057,17.837665,0.911111,9
1,draft_fp16_t1,The capital of France,4.592845,11.757418,0.493506,16
2,draft_fp16_t1,Explain the theory of,3.894068,13.096843,0.584615,13
3,draft_4bit,Write a Python functio,5.910618,8.459353,0.709091,11
4,draft_4bit,The capital of France,1.098486,1.820687,0.000000,2
5,draft_4bit,Explain the theory of,6.372203,8.003511,0.727273,11


#### 2. LoRA Distillation

In [41]:
def make_prompts(n=500):
    topics = ["math", "python", "history", "biology", "economics", "literature", "physics", "travel", "cooking", "ml"]
    tasks = [
        "Explain in simple terms:",
        "Write a short step-by-step guide for:",
        "Give 5 bullet points about:",
        "Provide a concise definition of:",
        "Answer with an example:",
    ]
    prompts = []
    for _ in range(n):
        a = np.random.choice(tasks)
        t = np.random.choice(topics)
        prompts.append(f"{a} {t}")
    return prompts

In [42]:
def teacher_generate_answer(prompt, max_new=128):
    prompt_text = build_chat_prompt(prompt)
    inp = tokenizer(prompt_text, return_tensors="pt").to(device)
    with torch.no_grad():
        out = target_model.generate(
            **inp,
            do_sample=False,
            temperature=None,       
            top_p=None,               
            top_k=None, 
            max_new_tokens=max_new,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    gen_ids = out[0, inp.input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True)

In [43]:
N_EX = 500
prompts_lora = make_prompts(N_EX)

In [44]:
pairs = []
for i, p in enumerate(prompts_lora, 1):
    ans = teacher_generate_answer(p, max_new=96)
    pairs.append({"prompt": p, "answer": ans})
    if i % 100 == 0:
        print(f"done {i}/{N_EX}")

done 100/500
done 200/500
done 300/500
done 400/500
done 500/500


In [45]:
draft_hf = AutoModelForCausalLM.from_pretrained(
    DRAFT_ID,
    torch_dtype=torch.float16,
    attn_implementation="sdpa",
).to(device)
draft_hf.train()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [46]:
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.04,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

In [47]:
draft_lora = get_peft_model(draft_hf, lora_cfg)
draft_lora.print_trainable_parameters()

trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497


In [48]:
class DistillSFTDataset(Dataset):
    def __init__(self, pairs, max_len=512):
        self.pairs = pairs
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        p = self.pairs[idx]["prompt"]
        a = self.pairs[idx]["answer"]

        prompt_text = build_chat_prompt(p)
        full_text = prompt_text + a

        prompt_ids = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=False).input_ids[0]
        full_ids = tokenizer(full_text, return_tensors="pt", add_special_tokens=False).input_ids[0]

        full_ids = full_ids[: self.max_len]
        prompt_len = min(len(prompt_ids), len(full_ids))

        labels = full_ids.clone()
        labels[:prompt_len] = -100

        return {"input_ids": full_ids, "labels": labels}

In [49]:
def collate(batch):
    pad_id = tokenizer.eos_token_id if tokenizer.pad_token_id is None else tokenizer.pad_token_id
    maxlen = max(x["input_ids"].shape[0] for x in batch)

    input_ids = torch.full((len(batch), maxlen), pad_id, dtype=torch.long)
    labels = torch.full((len(batch), maxlen), -100, dtype=torch.long)

    for i, ex in enumerate(batch):
        l = ex["input_ids"].shape[0]
        input_ids[i, :l] = ex["input_ids"]
        labels[i, :l] = ex["labels"]

    return {"input_ids": input_ids.to(device), "labels": labels.to(device)}

In [51]:
ds = DistillSFTDataset(pairs, max_len=512)
dl = DataLoader(ds, batch_size=1, shuffle=True, collate_fn=collate)

In [54]:
optim = torch.optim.AdamW(draft_lora.parameters(), lr=2e-4)

In [59]:
grad_accum = 10
max_steps = 500
scaler = torch.amp.GradScaler("cuda", enabled=(device == "cuda"))

In [63]:
draft_lora.train()
optim.zero_grad(set_to_none=True)

In [64]:
step = 0
t0 = time.time()

for batch in dl:
    if device == "cuda":
        with torch.amp.autocast("cuda", dtype=torch.float16):
            out = draft_lora(**batch)
            loss = out.loss / grad_accum
        scaler.scale(loss).backward()

        if (step + 1) % grad_accum == 0:
            scaler.step(optim)
            scaler.update()
            optim.zero_grad(set_to_none=True)
    else:
        out = draft_lora(**batch)
        loss = out.loss / grad_accum
        loss.backward()

        if (step + 1) % grad_accum == 0:
            optim.step()
            optim.zero_grad(set_to_none=True)

    if (step + 1) % 100 == 0:
        print(f"step {step+1}/{max_steps} | loss={loss.item()*grad_accum:.4f} | elapsed={time.time()-t0:.1f}s")

    step += 1
    if step >= max_steps:
        break

step 100/500 | loss=0.5005 | elapsed=30.8s
step 200/500 | loss=0.4576 | elapsed=59.8s
step 300/500 | loss=0.0274 | elapsed=88.9s
step 400/500 | loss=0.0031 | elapsed=118.1s
step 500/500 | loss=0.0065 | elapsed=148.6s


In [65]:
draft_lora.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.04, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(


In [66]:
draft_lora_logits = LogitsOnlyWrapper(draft_lora).eval()

In [67]:
eval_prompts = [
    "Explain what RMSNorm is.",
    "Give a short code example of a Python decorator.",
    "Why does RoPE help with long context?",
    "Write 3 tips for debugging PyTorch models.",
]

In [68]:
def acceptance_rate_on(prompts, draft_logits_model, K=5, max_new=64):
    acc = []
    for p in prompts:
        _, stats = speculative_sampling(p, max_new, target_model, draft_logits_model, tokenizer, K=K)
        acc.append(stats["total_accepted"] / max(1, stats["total_drafted"]))
    return sum(acc) / len(acc), acc

In [69]:
base_mean, base_list = acceptance_rate_on(eval_prompts, draft_model, K=5, max_new=64)
lora_mean, lora_list = acceptance_rate_on(eval_prompts, draft_lora_logits, K=5, max_new=64)

In [149]:
print(f"Baseline NanoQwen AR: {base_mean:6f}, {base_list}")
print(f"LoRA Draft AR: {lora_mean:6f}, {lora_list}")

Baseline NanoQwen AR: 0.571549, [0.44, 0.8153846153846154, 0.41904761904761906, 0.611764705882353]
LoRA Draft AR: 0.581818, [0.42727272727272725, 0.9, 0.2714285714285714, 0.7285714285714285]


#### 3. Layer Pruning Distillation

In [72]:
teacher = target_model
teacher.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [73]:
student_cfg = copy.deepcopy(teacher.config)
student_cfg.num_hidden_layers = teacher.config.num_hidden_layers // 2

In [74]:
student = AutoModelForCausalLM.from_config(student_cfg).to(device).half()
student.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-13): 14 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [75]:
t_sd = teacher.state_dict()
s_sd = student.state_dict()

In [76]:
def detect_layer_prefix(sd_keys):
    if any(k.startswith("model.layers.") for k in sd_keys):
        return "model.layers."
    if any(k.startswith("layers.") for k in sd_keys):
        return "layers."
    raise ValueError("err detect layer prefix")

In [77]:
t_pref = detect_layer_prefix(t_sd.keys())
s_pref = detect_layer_prefix(s_sd.keys())

In [78]:
new_sd = {}

In [80]:
for k, v in t_sd.items():
    if k.startswith(t_pref):
        continue
    if k in s_sd and s_sd[k].shape == v.shape:
        new_sd[k] = v

In [81]:
s_n = student_cfg.num_hidden_layers

In [82]:
for i_s in range(s_n):
    i_t = i_s * 2
    for k_s in list(s_sd.keys()):
        if not k_s.startswith(f"{s_pref}{i_s}."):
            continue
        k_tail = k_s.split(f"{s_pref}{i_s}.", 1)[1]
        k_t = f"{t_pref}{i_t}." + k_tail
        if k_t in t_sd and s_sd[k_s].shape == t_sd[k_t].shape:
            new_sd[k_s] = t_sd[k_t]

In [83]:
missing, unexpected = student.load_state_dict(new_sd, strict=False)
print("Missing:", len(missing), "Unexpected:", len(unexpected))

Missing: 0 Unexpected: 0


In [87]:
student.train()
lora_kd_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

In [88]:
student_lora = get_peft_model(student, lora_kd_cfg).to(device)
student_lora.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-13): 14 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

In [89]:
kd_prompts = make_prompts(500)

In [90]:
class KDPromptDataset(Dataset):
    def __init__(self, prompts, max_len=256):
        self.prompts = prompts
        self.max_len = max_len
    def __len__(self):
        return len(self.prompts)
    def __getitem__(self, idx):
        p = self.prompts[idx]
        txt = build_chat_prompt(p)
        ids = tokenizer(txt, return_tensors="pt", add_special_tokens=False).input_ids[0][:self.max_len]
        return ids

In [115]:
def kd_collate(batch):
    pad_id = tokenizer.eos_token_id if tokenizer.pad_token_id is None else tokenizer.pad_token_id
    maxlen = max(x.shape[0] for x in batch)

    input_ids = torch.full((len(batch), maxlen), pad_id, dtype=torch.long)
    attn = torch.zeros((len(batch), maxlen), dtype=torch.long)

    for i, ids in enumerate(batch):
        l = ids.shape[0]
        input_ids[i, :l] = ids
        attn[i, :l] = 1

    return {
        "input_ids": input_ids.to(device),
        "attention_mask": attn.to(device),
    }

In [117]:
kd_ds = KDPromptDataset(kd_prompts, max_len=256)
kd_dl = DataLoader(kd_ds, batch_size=1, shuffle=True, collate_fn=kd_collate)

In [119]:
def kd_loss(student_logits, teacher_logits, attention_mask, T=2.0):
    s = student_logits / T
    t = teacher_logits / T
    logp_s = torch.log_softmax(s, dim=-1)
    p_t = torch.softmax(t, dim=-1)

    kl = torch.sum(p_t * (torch.log(p_t + 1e-8) - logp_s), dim=-1)
    kl = kl * attention_mask
    return kl.sum() / attention_mask.sum() * (T * T)

In [121]:
optim = torch.optim.AdamW(student_lora.parameters(), lr=2e-4)
scaler = torch.amp.GradScaler("cuda", enabled=(device == "cuda"))

In [123]:
max_steps = 500
t0 = time.time()

teacher.eval()
student_lora.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-13): 14 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

In [125]:
for step, batch in enumerate(kd_dl, 1):
    if device == "cuda":
        with torch.no_grad(), torch.amp.autocast("cuda", dtype=torch.float16):
            t_out = teacher(**batch)
            t_logits = t_out.logits.detach()

        with torch.amp.autocast("cuda", dtype=torch.float16):
            s_out = student_lora(**batch)
            s_logits = s_out.logits
            loss = kd_loss(s_logits, t_logits, batch["attention_mask"], T=2.0)

        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()
        optim.zero_grad(set_to_none=True)
    else:
        with torch.no_grad():
            t_out = teacher(**batch)
            t_logits = t_out.logits.detach()

        s_out = student_lora(**batch)
        s_logits = s_out.logits
        loss = kd_loss(s_logits, t_logits, batch["attention_mask"], T=2.0)

        loss.backward()
        optim.step()
        optim.zero_grad(set_to_none=True)

    if step % 100 == 0:
        print(f"step {step}/{max_steps} | kd_loss={loss.item():.4f} | elapsed={time.time()-t0:.1f}s")

    if step >= max_steps:
        break

step 100/500 | kd_loss=1.3122 | elapsed=24.6s
step 200/500 | kd_loss=0.5965 | elapsed=49.1s
step 300/500 | kd_loss=0.3884 | elapsed=73.3s
step 400/500 | kd_loss=0.4853 | elapsed=97.9s
step 500/500 | kd_loss=0.2027 | elapsed=121.8s


In [126]:
student_lora.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-13): 14 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

In [127]:
pruned_kd_draft = LogitsOnlyWrapper(student_lora).eval()

In [128]:
base_mean, _ = acceptance_rate_on(eval_prompts, draft_model, K=5, max_new=64)
pruned_mean, _ = acceptance_rate_on(eval_prompts, pruned_kd_draft, K=5, max_new=64)

In [151]:
print(f"Baseline NanoQwen AR: {base_mean:6f}")
print(f"Pruned+KD Draft AR: {pruned_mean:6f}")

Baseline NanoQwen AR: 0.571549
Pruned+KD Draft AR: 0.021690


In [130]:
rows_pruned = run_spec_bench(pruned_kd_draft, "draft_prunedKD", K=5)
df_t3 = pd.DataFrame(rows_pruned, columns=["Draft","Prompt","Time(s)","Spec tok/s","AcceptRate","TargetCalls"])
df_t3

,Draft,Prompt,Time(s),Spec tok/s,AcceptRate,TargetCalls
0,draft_prunedKD,Write a Python functio,18.314411,2.730090,0.008333,48
1,draft_prunedKD,The capital of France,0.793908,2.519183,0.000000,2
2,draft_prunedKD,Explain the theory of,17.374268,2.877819,0.022222,45
